In [148]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import mysql.connector
import pandas as pd
from datetime import datetime

In [154]:
# CVS 파일 읽기
df = pd.read_excel("../data/가계신용_용도별.xlsx")

df= df.transpose()
df.reset_index(inplace=True)
df = df.drop([0, 1, 3, 4, 5], axis=1)

# 결측값 처리
df.loc[df[2]=='-'] = None
df = df.dropna(axis=0)
df = df.drop(df.index[0])

df_mortgage_loan = df.reset_index(drop=True)
df_mortgage_loan.columns = ['date', 'mortgage_loan']

/home/naon/dev_ws/eda_repo/.venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [163]:
# 분기를 월로 매핑하는 딕셔너리
quarter_to_month = {
    '1': '01',  # 1/4분기 -> 1월
    '2': '04',  # 2/4분기 -> 4월
    '3': '07',  # 3/4분기 -> 7월
    '4': '10'   # 4/4분기 -> 10월
}

# 연도와 분기 추출 및 변환
def convert_quarter_to_date(row):
    year, quarter = row.split('.')  # '2007.4/4'에서 연도와 분기 구분
    quarter = quarter.split('/')[0]  # '4/4'에서 '4' 추출
    month = quarter_to_month[quarter]  # 분기에 따라 월 매핑
    return f"{year}-{month}-01"  # 'YYYY-MM-01' 형식으로 변환

# 형식 변환
df_mortgage_loan['date'] = df_mortgage_loan['date'].apply(convert_quarter_to_date)
df_mortgage_loan['mortgage_loan'] = df_mortgage_loan['mortgage_loan'].astype(float)

# 데이터 타입을 datetime으로 변환
df_mortgage_loan['date'] = pd.to_datetime(df_mortgage_loan['date'])

# 데이터프레임을 엑셀 파일로 저장
df_mortgage_loan.to_excel("../data/mortgage_loan.xlsx", index=False, engine='openpyxl')
print(df_mortgage_loan)

         date  mortgage_loan
0  2007-10-01       343798.9
1  2008-01-01       346945.6
2  2008-04-01       355012.2
3  2008-07-01       362499.8
4  2008-10-01       369163.3
..        ...            ...
62 2023-04-01      1031829.9
63 2023-07-01      1049091.6
64 2023-10-01      1064339.1
65 2024-01-01      1076697.3
66 2024-04-01      1092722.5

[67 rows x 2 columns]


In [165]:
import mysql.connector

conn = mysql.connector.connect(
    host = "eda-database.c3oek28e8yta.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "whskdhs",
    password = "whskdhs1234",
    database = "workspace"
)

cursor = conn.cursor(buffered=True)

df = pd.read_excel('../data/mortgage_loan.xlsx')

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           67 non-null     datetime64[ns]
 1   mortgage_loan  67 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.2 KB


In [167]:
# 데이터베이스에 저장
def insert_data_to_mortgage_loan(conn, cursor, df):
    try:
        for i, row, in df.iterrows():
            sql_loan = "INSERT INTO mortgage_loan VALUES (%s, %s)"
            values_loan = (row['date'], row['mortgage_loan'])
            cursor.execute(sql_loan, values_loan)
            print(f"Inserted row {i+1} into article: {row['date']}")
            conn.commit()
    
    except mysql.connector.Error as error:
        print(f"Error inserting data: {error}")

In [168]:
insert_data_to_mortgage_loan(conn, cursor, df)

Inserted row 1 into article: 2007-10-01 00:00:00
Inserted row 2 into article: 2008-01-01 00:00:00
Inserted row 3 into article: 2008-04-01 00:00:00
Inserted row 4 into article: 2008-07-01 00:00:00
Inserted row 5 into article: 2008-10-01 00:00:00
Inserted row 6 into article: 2009-01-01 00:00:00
Inserted row 7 into article: 2009-04-01 00:00:00
Inserted row 8 into article: 2009-07-01 00:00:00
Inserted row 9 into article: 2009-10-01 00:00:00
Inserted row 10 into article: 2010-01-01 00:00:00
Inserted row 11 into article: 2010-04-01 00:00:00
Inserted row 12 into article: 2010-07-01 00:00:00
Inserted row 13 into article: 2010-10-01 00:00:00
Inserted row 14 into article: 2011-01-01 00:00:00
Inserted row 15 into article: 2011-04-01 00:00:00
Inserted row 16 into article: 2011-07-01 00:00:00
Inserted row 17 into article: 2011-10-01 00:00:00
Inserted row 18 into article: 2012-01-01 00:00:00
Inserted row 19 into article: 2012-04-01 00:00:00
Inserted row 20 into article: 2012-07-01 00:00:00
Inserted 